In [ ]:
from helper import*
import params as cfg
import sys
sys.path.insert(1, '/Users/joshuaighalo/Documents/GitHub/eegDementia')

In [ ]:
"Functions utilized in this script"
def multipleFilesNotch(input_3d,fs):
    notch_out = []
    filtering = filters()
    for i in range(input_3d.shape[0]):
        notch_out.append(filtering.notch(input_3d[i,:,:],fs))
    return np.array(notch_out)
def multipleFilesbutterBandpass(input_3d,lowcut,highcut,fs):
    butter_out = []
    filtering = filters()
    for i in range(input_3d.shape[0]):
        butter_out.append(filtering.butterBandPass(input_3d[i,:,:],lowcut,highcut,fs))
    return np.array(butter_out)
def multipleArtfRemoval(input_3d,threshold_value,timearray,len_window,step_size,choice_numwindows,channels,group):
    print(group)
    artf_out = []
    for i in range(input_3d.shape[0]):
        artf_out.append(ar_maximumgradient(input_3d[i,:,:],threshold_value,timearray,len_window,step_size,choice_numwindows,channels))
    print('***************************************************************************************************************************************************************')
    return np.array(artf_out)
def multiplePSD(data_4D,fs):
    freqs,psd = [],[]
    for i in range(data_4D.shape[0]):
        freqs.append(computePSD(data_4D[i],fs,data_type='3D')[0])
        psd.append(computePSD(data_4D[i],fs,data_type='3D')[1])
    psd = np.array(psd)
    freqs = np.array(freqs)
    return freqs,psd
def multipleRelativeBandPower(input_3d,freq_array,low,high):
    avg_out = []
    for i in range(input_3d.shape[0]):
        avg_out.append(psdRelativeBandPower(input_3d[i,:,:],freq_array,low,high))
    return np.array(avg_out)
def spectrogram4D(data,title,fs,figsize,subtitles):
    eeg,sr = data,fs
    WinLength,step = int(sr/2),int(sr/4)
    myparams = dict(nperseg = WinLength, noverlap = WinLength-step, return_onesided=True, mode='magnitude')
    f_1, nseg_1, Sxx_1 = signal.spectrogram(x = eeg[:,0], fs=sr, **myparams)
    f_2, nseg_2, Sxx_2 = signal.spectrogram(x = eeg[:,1], fs=sr, **myparams)
    f_3, nseg_3, Sxx_3 = signal.spectrogram(x = eeg[:,2], fs=sr, **myparams)
    f_4, nseg_4, Sxx_4 = signal.spectrogram(x = eeg[:,3], fs=sr, **myparams)
    fig, ax = plt.subplots(1,4, figsize=(figsize[0],figsize[1]), constrained_layout=True)
    fig.suptitle(title)
    ax[0].set_title(subtitles[0])
    ax[1].set_title(subtitles[1])
    ax[2].set_title(subtitles[2])
    ax[3].set_title(subtitles[3])
    ax[0].set(xlabel='Time (ms)',ylabel='Frequency (Hz)')
    ax[1].set(xlabel='Time (ms)',ylabel='Frequency (Hz)')
    ax[2].set(xlabel='Time (ms)',ylabel='Frequency (Hz)')
    ax[3].set(xlabel='Time (ms)',ylabel='Frequency (Hz)')
    #nseg_1,nseg_2,nseg_3,nseg_4 = np.linspace(0,500,4),np.linspace(0,500,4),np.linspace(0,500,4),np.linspace(0,500,4)
    nseg_1,nseg_2,nseg_3,nseg_4 = np.linspace(0,500,len(Sxx_1.T)),np.linspace(0,500,len(Sxx_2.T)),np.linspace(0,500,len(Sxx_3.T)),np.linspace(0,500,len(Sxx_4.T))
    X1,X2,X3,X4 = nseg_1,nseg_2,nseg_3,nseg_4
    Y1,Y2,Y3,Y4 = f_1,f_2,f_3,f_4
    Z1,Z2,Z3,Z4 = Sxx_1,Sxx_2,Sxx_3,Sxx_4
    levels = 45
    spectrum = ax[0].contourf(X1,Y1,Z1,levels, cmap='jet')#,'linecolor','none')
    spectrum = ax[1].contourf(X2,Y2,Z2,levels, cmap='jet')#,'linecolor','none')
    spectrum = ax[2].contourf(X3,Y3,Z3,levels, cmap='jet')#,'linecolor','none')
    spectrum = ax[3].contourf(X4,Y4,Z4,levels, cmap='jet')#,'linecolor','none')
    cbar = plt.colorbar(spectrum)#, boundaries=np.linspace(0,1,5))
    #cbar.ax.set_ylabel('Amplitude (dB)', rotation=90)
    plt.show()
    return X1,X2,X3,X4,Y1,Y2,Y3,Y4,Z1,Z2,Z3,Z4
def stft_plots(data,fig_size,nfft,sfreq,noverlap,title,subtitles):
    fig, ax = plt.subplots(1,4, figsize=(fig_size[0],fig_size[1]), constrained_layout=True)
    d1,f1,t1,im1 = plt.specgram(data[:,0], NFFT= nfft, Fs=sfreq, noverlap=noverlap)
    d2,f2,t2,im2 = plt.specgram(data[:,1], NFFT= nfft, Fs=sfreq, noverlap=noverlap)
    d3,f3,t3,im3 = plt.specgram(data[:,2], NFFT= nfft, Fs=sfreq, noverlap=noverlap) 
    d4,f4,t4,im4 = plt.specgram(data[:,3], NFFT= nfft, Fs=sfreq, noverlap=noverlap)
    fig.suptitle(title)
    ax[0].set_title(subtitles[0])
    ax[1].set_title(subtitles[1])
    ax[2].set_title(subtitles[2])
    ax[3].set_title(subtitles[3])
    ax[0].set(xlabel='Time (s)',ylabel='Frequency (Hz)')
    ax[1].set(xlabel='Time (s)',ylabel='Frequency (Hz)')
    ax[2].set(xlabel='Time (s)',ylabel='Frequency (Hz)')
    ax[3].set(xlabel='Time (s)',ylabel='Frequency (Hz)')
    ax[0].set_ylim(0,80)
    ax[1].set_ylim(0,80)
    ax[2].set_ylim(0,80)
    ax[3].set_ylim(0,80)
    t1,t2,t3,t4= np.linspace(0,500,len(d1.T)),np.linspace(0,500,len(d2.T)),np.linspace(0,500,len(d3.T)),np.linspace(0,500,len(d4.T))
    spectrum = ax[0].contourf(t1,f1,d1, cmap='jet')
    spectrum = ax[1].contourf(t2,f2,d2, cmap='jet')
    spectrum = ax[2].contourf(t3,f3,d3, cmap='jet') 
    spectrum = ax[3].contourf(t4,f4,d4, cmap='jet')
    cbar = plt.colorbar(spectrum)
    plt.show()
    return d1,d2,d3,d4,f1,f2,f3,f4,t1,t2,t3,t4

In [ ]:
"""
#   Group-Task-Time
M11 = ['m11_1.zip','m11_2.zip','m11_3.zip','m11_4.zip','m11_5.zip']
M21 = ['m21_1.zip','m21_2.zip','m21_3.zip','m21_4.zip','m21_5.zip']
M12 = ['m12_1.zip','m12_2.zip','m12_3.zip','m12_4.zip','m12_5.zip']
M22 = ['m22_1.zip','m22_2.zip','m22_3.zip','m22_4.zip','m22_5.zip']
C11 = ['c11_1.zip','c11_2.zip','c11_3.zip','c11_4.zip','c11_5.zip']
C21 = ['c21_1.zip','c21_2.zip','c21_3.zip','c21_4.zip','c21_5.zip']
C12 = ['c12_1.zip','c12_2.zip','c12_3.zip','c12_4.zip','c12_5.zip']
C22 = ['c22_1.zip','c22_2.zip','c22_3.zip','c22_4.zip','c22_5.zip']
localDir = "/Users/joshuaighalo/Downloads/v2/"
destDir = "/Users/joshuaighalo/Downloads/v2/dest/"
os.system('rm -rf %s/*' % destDir)
fs_setting = 'constant'
"""

In [ ]:
#"""
#   Group-Task-Time Ryan SEEN
M11 = ['m11_1.zip','m11_2.zip','m11_3.zip','m11_4.zip','m11_5.zip']
C21 = ['m21_1.zip','m21_2.zip','m21_3.zip','m21_4.zip','m21_5.zip']
M12 = ['m12_1.zip','m12_2.zip','m12_3.zip','m12_4.zip','m12_5.zip']
C22 = ['m22_1.zip','m22_2.zip','m22_3.zip','m22_4.zip','m22_5.zip']
C11 = ['c11_1.zip','c11_2.zip','c11_3.zip','c11_4.zip','c11_5.zip']
M21 = ['c21_1.zip','c21_2.zip','c21_3.zip','c21_4.zip','c21_5.zip']
C12 = ['c12_1.zip','c12_2.zip','c12_3.zip','c12_4.zip','c12_5.zip']
M22 = ['c22_1.zip','c22_2.zip','c22_3.zip','c22_4.zip','c22_5.zip']
localDir = "/Users/joshuaighalo/Downloads/v2/"
destDir = "/Users/joshuaighalo/Downloads/v2/dest/"
os.system('rm -rf %s/*' % destDir)
fs_setting = 'constant'
#"""

In [ ]:
#   Extract eeg data from participants within the groups
M11 = zipExtract(M11,localDir,destDir,"M11",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
M21 = zipExtract(M21,localDir,destDir,"M21",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
M12 = zipExtract(M12,localDir,destDir,"M12",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
M22 = zipExtract(M22,localDir,destDir,"M22",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C11 = zipExtract(C11,localDir,destDir,"C11",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C21 = zipExtract(C21,localDir,destDir,"C21",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C12 = zipExtract(C12,localDir,destDir,"C12",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
C22 = zipExtract(C22,localDir,destDir,"C22",cfg.fs,cfg.collection_time,cfg.fs_setting)[1]
time_s = np.arange(0,cfg.collection_time,1/cfg.fs)

In [ ]:
#   Notch filters data collected from participants
noc_M11 = multipleFilesNotch(M11,cfg.fs)
noc_M21 = multipleFilesNotch(M21,cfg.fs)
noc_M12 = multipleFilesNotch(M12,cfg.fs)
noc_M22 = multipleFilesNotch(M22,cfg.fs)
noc_C11 = multipleFilesNotch(C11,cfg.fs)
noc_C21 = multipleFilesNotch(C21,cfg.fs)
noc_C12 = multipleFilesNotch(C12,cfg.fs)
noc_C22 = multipleFilesNotch(C22,cfg.fs)

In [ ]:
#   Bandpass filters data collected from participants
band_M11 = multipleFilesbutterBandpass(noc_M11,cfg.lowcut,cfg.highcut,cfg.fs)
band_M21 = multipleFilesbutterBandpass(noc_M21,cfg.lowcut,cfg.highcut,cfg.fs)
band_M12 = multipleFilesbutterBandpass(noc_M12,cfg.lowcut,cfg.highcut,cfg.fs)
band_M22 = multipleFilesbutterBandpass(noc_M22,cfg.lowcut,cfg.highcut,cfg.fs)
band_C11 = multipleFilesbutterBandpass(noc_C11,cfg.lowcut,cfg.highcut,cfg.fs)
band_C21 = multipleFilesbutterBandpass(noc_C21,cfg.lowcut,cfg.highcut,cfg.fs)
band_C12 = multipleFilesbutterBandpass(noc_C12,cfg.lowcut,cfg.highcut,cfg.fs)
band_C22 = multipleFilesbutterBandpass(noc_C22,cfg.lowcut,cfg.highcut,cfg.fs)

In [ ]:
threshold_value,window_size,step_size,channels,choice_numwindows = 10,0.5,0.5,['TP9','AF7','AF8','TP10'],240
arM11 = multipleArtfRemoval(band_M11,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"M11")
arM21 = multipleArtfRemoval(band_M21,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"M21")
arM12 = multipleArtfRemoval(band_M12,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"M12")
arM22 = multipleArtfRemoval(band_M22,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"M22")
arC11 = multipleArtfRemoval(band_C11,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"C11")
arC21 = multipleArtfRemoval(band_C21,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"C21")
arC12 = multipleArtfRemoval(band_C12,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"C12")
arC22 = multipleArtfRemoval(band_C22,threshold_value,time_s,window_size,step_size,choice_numwindows,channels,"C22")

In [ ]:
arr_M11 = np.transpose(arM11,(0,1,3,2))
arr_M21 = np.transpose(arM21,(0,1,3,2))
arr_M12 = np.transpose(arM12,(0,1,3,2))
arr_M22 = np.transpose(arM22,(0,1,3,2))
arr_C11 = np.transpose(arC11,(0,1,3,2))
arr_C21 = np.transpose(arC21,(0,1,3,2))
arr_C12 = np.transpose(arC12,(0,1,3,2))
arr_C22 = np.transpose(arC22,(0,1,3,2))

In [ ]:
freqs,psd_M11 = multiplePSD(arr_M11,cfg.fs)
freqs,psd_M21 = multiplePSD(arr_M21,cfg.fs)
freqs,psd_M12 = multiplePSD(arr_M12,cfg.fs)
freqs,psd_M22 = multiplePSD(arr_M22,cfg.fs)
freqs,psd_C11 = multiplePSD(arr_C11,cfg.fs)
freqs,psd_C21 = multiplePSD(arr_C21,cfg.fs)
freqs,psd_C12 = multiplePSD(arr_C12,cfg.fs)
freqs,psd_C22 = multiplePSD(arr_C22,cfg.fs)
freqs = freqs[0,0,0,:]